In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carica il dataset
file_path = 'dati.tsv'
df = pd.read_csv(file_path, delimiter='\t')
df.rename(columns={'data_inizio_prove': 'data_campionamento'}, inplace=True)

# Cambia il 'punto_prelievo' per il codice LIMS specifico
df.loc[df['lims'] == '21LA13868', 'punto_prelievo'] = 'CARSEGGIO'

# Estrai le colonne rilevanti per l'analisi
df['data_campionamento'] = pd.to_datetime(df['data_campionamento'], format='%d/%m/%Y')
df['anno'] = df['data_campionamento'].dt.year
df['mese'] = df['data_campionamento'].dt.month
df['giorno'] = df['data_campionamento'].dt.date

# Filtra il dataframe per gli anni 2020, 2021, 2022, 2023
df_filtrato = df[df['anno'].isin([2020, 2021, 2022, 2023])].copy()

# Controlla che il numero di misure sia uguale
# Semplifica punto_prelievo alla sua parte iniziale per i punti specificati
df_filtrato['punto_prelievo'] = df_filtrato['punto_prelievo'].str.extract('(MORADUCCIO|VALSALVA|GATTO|CARSEGGIO|PRATO|CASTEL DEL RIO)', expand=False)

# Rimuovi duplicati
df_filtrato.drop_duplicates(subset=None, keep='first', inplace=True)

# Salva le misurazioni individuali di punto_prelievo in file CSV separati in ordine cronologico
lista_punti_prelievo = ['MORADUCCIO', 'VALSALVA', 'GATTO', 'CARSEGGIO', 'PRATO', 'CASTEL DEL RIO']
percorsi_csv = []
for punto_prelievo in lista_punti_prelievo:
    df_punto_prelievo = df_filtrato[df_filtrato['punto_prelievo'] == punto_prelievo].sort_values(by='data_campionamento')
    percorso_csv_punto_prelievo = f'{punto_prelievo.lower()}_misurazioni.csv'
    df_punto_prelievo[['data_campionamento', 'escherichia_coli', 'enterococchi']].to_csv(percorso_csv_punto_prelievo, index=False)
    percorsi_csv.append(percorso_csv_punto_prelievo)

percorsi_csv

# Sostituisci i valori zero e i valori inferiori a 1 con il limite minimo rilevabile per evitare problemi con log10
limite_minimo_rilevabile = 1
colonne_batteriche = ['escherichia_coli', 'enterococchi']
for colonna in colonne_batteriche:
    df_filtrato[colonna] = pd.to_numeric(df_filtrato[colonna], errors='coerce').fillna(limite_minimo_rilevabile)
    df_filtrato[colonna] = df_filtrato[colonna].replace(0, limite_minimo_rilevabile)
    df_filtrato[colonna] = df_filtrato[colonna].apply(lambda x: max(x, limite_minimo_rilevabile))

# Applica la trasformazione log10 alle colonne batteriche
for colonna in colonne_batteriche:
    df_filtrato[colonna] = np.log10(df_filtrato[colonna])

# Calcola la media (µ) e la deviazione standard (omega) dei valori log10 per ogni punto_prelievo
risultati = df_filtrato.groupby('punto_prelievo')[colonne_batteriche].agg(['mean', 'std']).reset_index()
risultati.columns = ['punto_prelievo', 'escherichia_coli_mean', 'escherichia_coli_std', 'enterococchi_mean', 'enterococchi_std']

# Calcola i percentili 90° e 95°
percentili = {}
for index, row in risultati.iterrows():
    punto = row['punto_prelievo']
    percentili[punto] = {}
    for colonna in colonne_batteriche:
        mean = row[f'{colonna}_mean']
        std = row[f'{colonna}_std']
        percentile_90 = 10**(mean + 1.282 * std)
        percentile_95 = 10**(mean + 1.65 * std)
        percentili[punto][f'{colonna}_90_percentile'] = percentile_90
        percentili[punto][f'{colonna}_95_percentile'] = percentile_95

# Converti i risultati in un DataFrame per una migliore leggibilità
df_percentili = pd.DataFrame(percentili).T.reset_index()

df_percentili.columns = ['punto_prelievo', 'escherichia_coli_90_percentile', 'escherichia_coli_95_percentile', 'enterococchi_90_percentile', 'enterococchi_95_percentile']

# Arrotonda i valori a una cifra decimale
df_percentili = df_percentili.round(1)

# Rinomina gli indici
dizionario_rinomina = {
    'MORADUCCIO': 'MORADUCCIO',
    'VALSALVA': 'LIDO_DI_VALSALVA',
    'GATTO': 'PONTE_GATTO_NERO',
    'CARSEGGIO': 'LIDO_DI_CARSEGGIO',
    'PRATO': 'LIDO_DI_PRATO',
    'CASTEL DEL RIO': 'PONTE_DEGLI_ALIDOSI'
}

df_percentili['punto_prelievo'] = df_percentili['punto_prelievo'].map(dizionario_rinomina)

# Riordina il DataFrame
ordine = ['MORADUCCIO', 'LIDO_DI_VALSALVA', 'PONTE_DEGLI_ALIDOSI', 'LIDO_DI_CARSEGGIO', 'LIDO_DI_PRATO', 'PONTE_GATTO_NERO']
df_percentili = df_percentili.set_index('punto_prelievo').loc[ordine].reset_index()

# Salva i risultati in un file CSV
percorso_csv_percentili = 'analisi_percentili.csv'
df_percentili.to_csv(percorso_csv_percentili, index=False)

# Salva i risultati in un file JPG
plt.figure(figsize=(12, 6))  # Aumenta le dimensioni della figura
plt.axis('off')
larghezze_colonne = [0.25, 0.3, 0.3, 0.3, 0.3]  # Aumenta le larghezze delle colonne
tbl = plt.table(cellText=df_percentili.values, colLabels=df_percentili.columns, colWidths=larghezze_colonne, cellLoc='center', loc='center')
tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1.2, 1.2)
plt.savefig('analisi_percentili.jpg', bbox_inches='tight', pad_inches=0.1)
plt.close()

# Definisci le soglie
soglie = {
    'escherichia_coli': {
        'eccellente': 500,
        'buona': 1000,
        'sufficiente': 900,

            },
    'enterococchi': {
        'eccellente': 200,
        'buona': 400,
        'sufficiente': 330,
    }
}
# Funzione per classificare la qualità dell'acqua
def classifica_qualita_acqua(row):
    if row['escherichia_coli_95_percentile'] <= soglie['escherichia_coli']['eccellente'] and \
       row['enterococchi_95_percentile'] <= soglie['enterococchi']['eccellente']:
        return 'Qualità eccellente'
    elif row['escherichia_coli_95_percentile'] <= soglie['escherichia_coli']['buona'] and \
         row['enterococchi_95_percentile'] <= soglie['enterococchi']['buona']:
        return 'Qualità buona'
    elif row['escherichia_coli_90_percentile'] <= soglie['escherichia_coli']['sufficiente'] and \
         row['enterococchi_90_percentile'] <= soglie['enterococchi']['sufficiente']:
        return 'Qualità sufficiente'
    else:
        return 'Qualità scarsa'

# Applica la classificazione
df_percentili['qualita'] = df_percentili.apply(classifica_qualita_acqua, axis=1)

# Output della classificazione
print(df_percentili[['punto_prelievo', 'qualita']])

# Tracciare la classificazione come una tabella
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
dati_tabella = df_percentili[['punto_prelievo', 'qualita']].values
tabella = ax.table(cellText=dati_tabella, colLabels=['Punto Prelievo', 'Qualità'], cellLoc='center', loc='center')
tabella.auto_set_font_size(False)
tabella.set_fontsize(12)
tabella.scale(1.5, 1.5)
plt.title("Classificazione Qualità delle Acque", fontsize=16)

# Salva la tabella come file JPG
plt.savefig('classificazione_qualita_acque.jpg', bbox_inches='tight', pad_inches=0.1)
plt.close()

# Salva la classificazione in un file CSV
df_percentili[['punto_prelievo', 'qualita']].to_csv('classificazione_qualita_acque.csv', index=False)


        punto_prelievo              qualita
0           MORADUCCIO  Qualità sufficiente
1     LIDO_DI_VALSALVA  Qualità sufficiente
2  PONTE_DEGLI_ALIDOSI  Qualità sufficiente
3    LIDO_DI_CARSEGGIO       Qualità scarsa
4        LIDO_DI_PRATO  Qualità sufficiente
5     PONTE_GATTO_NERO  Qualità sufficiente


In [102]:
# Verifica le date presenti in certi punto_prelievo e non in altri
lista_punti_prelievo = ['MORADUCCIO', 'VALSALVA', 'GATTO', 'CARSEGGIO', 'PRATO', 'CASTEL DEL RIO']
presenza_date = df_filtrato.groupby('punto_prelievo')['data_campionamento'].unique().reset_index()
dizionario_date = {row['punto_prelievo']: set(row['data_campionamento']) for index, row in presenza_date.iterrows()}

# Trova e stampa le date che sono presenti in certi punto_prelievo e non in altri
tutte_le_date = set(df_filtrato['data_campionamento'].unique())
for punto in lista_punti_prelievo:
    altri_punti = lista_punti_prelievo.copy()
    altri_punti.remove(punto)
    altre_date = set.union(*(dizionario_date[altro] for altro in altri_punti))
    date_uniche = dizionario_date[punto] - altre_date
    if date_uniche:
        print(f"Date presenti in {punto} e non in altri: {sorted(date_uniche)}")

# Verifica le date con misurazioni duplicate nello stesso punto_prelievo
duplicati = df_filtrato[df_filtrato.duplicated(subset=['data_campionamento', 'punto_prelievo'], keep=False)]

if not duplicati.empty:
    print("Misurazioni duplicate trovate nella stessa data e nello stesso punto di prelievo:")
    print(duplicati[['data_campionamento', 'punto_prelievo']])
else:
    print("Nessuna misurazione duplicata trovata nella stessa data e nello stesso punto di prelievo.")

# Raggruppa per punto_prelievo e anno, quindi conta il numero di misurazioni
conteggio_misurazioni = df_filtrato.groupby(['punto_prelievo', 'anno']).size().reset_index(name='conteggi')
print(conteggio_misurazioni)


Misurazioni duplicate trovate nella stessa data e nello stesso punto di prelievo:
    data_campionamento  punto_prelievo
216         2021-06-22  CASTEL DEL RIO
220         2021-06-22  CASTEL DEL RIO
    punto_prelievo  anno  conteggi
0        CARSEGGIO  2020        12
1        CARSEGGIO  2021         9
2        CARSEGGIO  2022        12
3        CARSEGGIO  2023         8
4   CASTEL DEL RIO  2020        12
5   CASTEL DEL RIO  2021        10
6   CASTEL DEL RIO  2022        12
7   CASTEL DEL RIO  2023         8
8            GATTO  2020        12
9            GATTO  2021         9
10           GATTO  2022        12
11           GATTO  2023         8
12      MORADUCCIO  2020        12
13      MORADUCCIO  2021         8
14      MORADUCCIO  2022        12
15      MORADUCCIO  2023         8
16           PRATO  2020        12
17           PRATO  2021         9
18           PRATO  2022        12
19           PRATO  2023         8
20        VALSALVA  2020        12
21        VALSALVA  2021       